# Lab02: Frequent itemset mining

- Student ID: 20127028
- Student name: Vo Van Hoang

**How to do your homework**


You will work directly on this notebook; the word `TODO` indicate the parts you need to do.

You can discuss ideas with classmates as well as finding information from the internet, book, etc...; but *this homework must be yours*.

**How to submit your homework**

Before submitting, rerun the notebook (`Kernel` ->` Restart & Run All`).

Then create a folder named `ID` (for example, if your ID is 1234567, then name the folder `1234567`) Copy file notebook to this folder, compress and submit it on moodle.

**Contents:**

- Frequent itemset mining.

# 1. Preliminaries
## This is how it all started ...
- Rakesh Agrawal, Tomasz Imielinski, Arun N. Swami: Mining Association Rules between Sets of Items in Large Databases. SIGMOD Conference 1993: 207-216
- Rakesh Agrawal, Ramakrishnan Srikant: Fast Algorithms for Mining Association Rules in Large Databases. VLDB 1994: 487-499

**These two papers are credited with the birth of Data Mining**
## Frequent itemset mining (FIM)

Find combinations of items (itemsets) that occur frequently.
## Applications
- Items = products, transactions = sets of products someone bought in one trip to the store.
$\Rightarrow$ items people frequently buy together.
    + Example: if people usually buy bread and coffee together, we run a sale of bread to attract people attention and raise price of coffee.
- Items = webpages, transactions = words. Unusual words appearing together in a large number of documents, e.g., “Brad” and “Angelina,” may indicate an interesting relationship.
- Transactions = Sentences, Items = Documents containing those sentences. Items that appear together too often could represent plagiarism.

## Transactional Database
A transactional database $D$ consists of $N$ transactions: $D=\left\{T_1,T_2,...,T_N\right\}$. A transaction $T_n \in D (1 \le n \le N)$ contains one or more items and that $I= \left\{ i_1,i_2,…,i_M \right\}$ is the set of distinct items in $D$, $T_n \subset I$. Commonly, a transactional database is represented by a flat file instead of a database system: items are non-negative integers, each row represents a transaction, items in a transaction separated by space.

Example: 

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

30 31 32 

33 34 35 

36 37 38 39 40 41 42 43 44 45 46 

38 39 47 48 

38 39 48 49 50 51 52 53 54 55 56 57 58 

32 41 59 60 61 62 

3 39 48 

63 64 65 66 67 68 



# Definition

- Itemset: A collection of one or more items.
    + Example: {1 4 5}
- **k-itemset**: An itemset that contains k items.
- Support: Frequency of occurrence of an itemset.
    + Example: From the example above, item 3 appear in 2 transactions so its support is 2.
- Frequent itemset: An itemset whose support is greater than or equal to a `minsup` threshold

# The Apriori Principle
- If an itemset is frequent, then all of its subsets must also be frequent.
- If an itemset is not frequent, then all of its supersets cannot be frequent.
- The support of an itemset never exceeds the support of its subsets.
$$ \forall{X,Y}: (X \subseteq Y) \Rightarrow s(X)\ge s(Y)$$


# 2. Implementation


## The Apriori algorithm
Suppose:

$C_k$ candidate itemsets of size k.

$L_k$ frequent itemsets of size k.

The level-wise approach of Apriori algorithm can be descibed as follow:
1. k=1, $C_k$ = all items.
2. While $C_k$ not empty:
    3. Scan the database to find which itemsets in $C_k$ are frequent and put them into $L_k$.
    4. Use $L_k$ to generate a collection of candidate itemsets $C_{k+1}$ of size k+1.
    5. k=k+1.

### Import library

In [309]:
from collections import defaultdict
import pandas as pd
import csv

### Read data
First we have to read data from database

In [310]:
def readData(path):
    """
    Parameters
    --------------------------
        path: path of database D.         
    --------------------------
    Returns
        data: a dictionary for representing database D
                 - keys: transaction tids
                 - values: itemsets.
        s: support of distict items in D.
    """
    data={}
    s=defaultdict(lambda: 0) # Initialize a dictionary for storing support of items in I.  
    with open(path,'rt') as f:
        tid=1;
        for line in f:
            itemset=set(map(int,line.split())) # a python set is a native way for storing an itemset.
            for item in itemset:  
                s[item]+=1     #Why don't we compute support of items while reading data?
            data[tid]= itemset
            tid+=1
    
    return data, s

### Tree Projection

**I gave you pseudo code of Apriori algorithm above but we implement Tree Projection. Tell me the differences of two algorithms.**


**TODO:**

In [311]:
def joinset(a, b):
    '''
    Parameters
    -------------------
        2 itemsets a and b (of course they are at same branch in search space)
    -------------------
    return
        ret: itemset generated by joining a and b
    '''
    # TODO (hint: this function will be called in generateSearchSpace method.):
    ret = list(set(a + b))
    ret = sorted(ret)  
    return ret

class TP:
    def __init__(self, data=None, s=None, minSup=None):
        self.data = data
        self.s = {}
        for key, support in sorted(s.items(), key=lambda item: item[1]):
            self.s[key] = support
        # TODO: why should we do this, answer it at the markdown below?
        self.minSup = minSup
        self.L = {}  # Store frequent itemsets mined from database
        self.runAlgorithm()
        
    def initialize(self):
        """
        Initialize search space at first step
        --------------------------------------
        We represent our search space in a tree structure
        """
        tree = {}
        search_space = {}
        for item, support in self.s.items():
            search_space[item] = {}
            search_space[item]['itemset'] = [item]
            ''' 
            python set does not remain elements order
            so we use a list to extend it easily when create new itemset 
            but why we store itemset in data by a python set???? '''
            # TODO: study about python set and its advantages,
            # answer at the markdown below.
            search_space[item]['pruned'] = False
            # TODO:
            # After finish implementing the algorithm tell me why should you use this
            # instead of delete item directly from search_space and tree.
            search_space[item]['support'] = support
            tree[item] = {}
            '''
            Why should i store an additional tree (here it called tree)? 
            Answer: This really help in next steps.
            Remember that there is always a big gap from theory to practicality
            and implementing this algorithm in python is not as simple as you think.
            '''
        return tree, search_space
    
    def computeItemsetSupport(self, itemset):
        '''Return support of itemset'''
        # TODO (hint: this is why i use python set in data)
        support = 0
        if isinstance(itemset, int):
            itemset = {itemset}
        else:
            itemset = set(itemset)
        for transaction in self.data.values():
            if itemset.issubset(transaction):
                support += 1
        return support
    
    def get_sub_tree(self, k, tree, search_space, itter_node):
        if k == 0:
            return search_space[itter_node]['support']
        subtree = search_space[itter_node]
        for node in subtree.keys():
            k-=1
            self.get_sub_tree(k,tree,search_space,node)

    def prune(self, k, tree, search_space):
        '''
        In this method we will find out which itemset in current search space is frequent
        itemset then add it to L[k]. In addition, we prune those are not frequent itemsets.
        '''
        if self.L.get(k) is None: self.L[k] = []
        # TODO
        itemset_support = []
        for it_set in search_space.keys():
            support = self.computeItemsetSupport(it_set)
            itemset_support.append(support)
        for itemset, properties in search_space.items():  
            if properties['pruned']:
                continue
            support = self.computeItemsetSupport(search_space[itemset]['itemset'])
            if support < self.minSup:
                search_space[itemset]['pruned'] = True
            else:
                frequent_itemset = list(properties['itemset'])
                self.L[k].append(frequent_itemset)
        
    def generateSearchSpace(self, k, tree, search_space):
        '''
        Generate search space for exploring k+1 itemset. (Recursive function)
        '''
        items = list(tree.keys())
        ''' print search_space.keys() you will understand  
         why we need an additional tree, '''
        l = len(items)
        self.prune(k, tree, search_space)
        if l == 0: return  # Stop condition
        for i in range(l - 1):
            sub_search_space = {}
            sub_tree = {}
            a = items[i]
            if search_space[a]['pruned']: continue
            for j in range(i + 1, l):
                b = items[j]
                search_space[a][b] = {}
                tree[a][b] = {}
                # You really need to understand what am i doing here before doing work below.
                # (Hint: draw tree and search space to draft).
                # TODO:
                # First create newset using join set               
                newset = tuple(sorted(joinset(list(search_space[a]['itemset']), list(search_space[b]['itemset']))))               
                # Second add newset to search_space
                if newset is not None:
                    newset_support = self.computeItemsetSupport(newset)
                    newset = tuple(newset)   
                    if newset_support >= self.minSup:
                        search_space[a][b]['itemset'] = list(newset)
                        search_space[a][b]['pruned'] = False
                        search_space[a][b]['support'] = newset_support
                        search_space[newset] = {'itemset': newset, 'pruned': False, 'support': newset_support}
                        sub_search_space[newset] = {'itemset': newset, 'pruned': False, 'support': newset_support}
                        sub_tree[newset] = {}                
            #  Generate search_space for k+1-itemset
            self.generateSearchSpace(k + 1, sub_tree, sub_search_space)
            
    def runAlgorithm(self):
        tree, search_space = self.initialize()  # generate search space for 1-itemset
        self.generateSearchSpace(1, tree, search_space)
        
    def miningResults(self):
        return self.L

Ok, let's test on a typical dataset `chess`.

In [312]:
data, s= readData('chess.txt')

In [313]:
#
a=TP(data=data,s=s, minSup=3000)
print(a.miningResults())

{1: [[48], [56], [66], [34], [62], [7], [36], [60], [40], [29], [52], [58]], 2: [[48, 52], [48, 58], [29, 56], [52, 56], [56, 58], [60, 66], [29, 66], [52, 66], [58, 66], [34, 40], [29, 34], [34, 52], [34, 58], [60, 62], [40, 62], [29, 62], [52, 62], [58, 62], [7, 60], [7, 40], [7, 29], [7, 52], [7, 58], [36, 60], [36, 40], [29, 36], [36, 52], [36, 58], [40, 60], [29, 60], [52, 60], [58, 60], [29, 40], [40, 52], [40, 58], [29, 52], [29, 58], [52, 58]], 3: [[48, 52, 58], [29, 52, 56], [29, 56, 58], [52, 56, 58], [29, 60, 66], [52, 60, 66], [58, 60, 66], [29, 52, 66], [29, 58, 66], [52, 58, 66], [29, 34, 40], [34, 40, 52], [34, 40, 58], [29, 34, 52], [29, 34, 58], [34, 52, 58], [29, 60, 62], [52, 60, 62], [58, 60, 62], [29, 40, 62], [40, 52, 62], [40, 58, 62], [29, 52, 62], [29, 58, 62], [52, 58, 62], [7, 40, 60], [7, 29, 60], [7, 52, 60], [7, 58, 60], [7, 29, 40], [7, 40, 52], [7, 40, 58], [7, 29, 52], [7, 29, 58], [7, 52, 58], [36, 40, 60], [29, 36, 60], [36, 52, 60], [36, 58, 60], [29

### Answer questions here:

**1. Why don't we compute support of items while reading data?**

- This method requires us to store all the transactions in memory which can become very out of memory or time-consuming as the size of the data file rises. Furthermore, we won't count item support while reading the data because we need to convert the raw data from string to int and then implement item support on transactions of data file.

**2. Why should we do sort?**
- Because it enables us to identify all of the individual items that appear in the data, count the frequency of occurrence of each item, and group all occurrences of the same item together, we can count each item more quickly and efficiently. Furthermore, data sorting aids in the generation of larger item sets from smaller item sets, which is required in order to locate all frequent item sets. A good way to narrow down the search space.

**3. Study about python set and its advantages?**

- To begin, any duplicates are automatically removed. Second, find the element that is shared by both sets, then remove the element from this set that is also shared by another set because it performs set operations. Third, Set Data contains only unique elements, making set operations like union, intersection be simpler. Fourth, tuples store their elements in hash tables, which allows for quick lookups. As a result, the data format in the FIM algorithm is set to help them perform efficiently in all aspects of time to realize capacity and memory usage.

**4. After finishing implementing the algorithm tell me why should you use this instead of deleting item directly from search_space and tree.**
- Using the variable pruned instead of directly deleting from search space and tree can assist us in keeping track of the items pruned during the tree traversal. This can be useful in certain situations, such as when analyzing the algorithm's performance or extracting information from the pruned items.


- Furthermore, removing an item directly can be computationally expensive because it necessitates a complete scan of the search space and tree. In contrast, the algorithm only needs to scan a subset of the search space and tree, saving significant computation time.

**5. Apriori algorithm and Tree Projection, tell me the differences of two algorithms.**
- Although these 2 algorithms have many similarities: to begin, both techniques are used to extract frequent itemets from a transaction database. Second, both techniques employ the concept of support to identify frequently occurring itemsets. Furthermore, both techniques involve the creation of a data structure that stores information about frequently occurring itemsets and can be used to efficiently generate rules. Furthermore, both techniques include a pruning step to eliminate infrequent itemets and reduce the search space. Finally, both techniques can be used to generate association rules from frequently used itemsets.


- Besides that, they also have some differences: First of all, The Tree Projection method uses less memory than the Apriori algorithm because it only keeps track of the frequently occurring items in the tree structure, whereas the Apriori algorithm must keep all candidate itemets in memory. Furthermore, because it can handle a broader range of data formats, the Apriori algorithm is more versatile than the Tree Projection method. Last but not least, implemented techniques of them are different: The Apriori algorithm employs a candidate generation approach, whereas the Tree Projection method traverses the data set and generates frequent itemsets using a tree-based approach.

# 3. Churn analysis

In this section, you will use frequent itemset mining technique to analyze `churn` dataset (for any purposes). 

* Remember this dataset is not represented as a transactional database, first thing that you have to do is transforming it into a flat file.

In [314]:
churn = []
with open('churn.txt', 'r') as temp:
    for line in temp:
        if 'blob-code blob-code-inner js-file-line' in line:
            start = line.find('>') + 1
            end = line.rfind('<')
            churn.append(line[start:end])
with open('churn.csv', 'w') as temp:
    temp.write('\n'.join(churn))
churn_df = pd.read_csv('churn.csv')
churn_df

,State,Account Length,Area Code,Phone,Int&#39;l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False.
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False.
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False.
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False.
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,AZ,192,415,414-4276,no,yes,36,156.2,77,26.55,...,126,18.32,279.1,83,12.56,9.9,6,2.67,2,False.
3329,WV,68,415,370-3271,no,no,0,231.1,57,39.29,...,55,13.04,191.3,123,8.61,9.6,4,2.59,3,False.
3330,RI,28,510,328-8230,no,no,0,180.8,109,30.74,...,58,24.55,191.9,91,8.64,14.1,6,3.81,2,False.
3331,CT,184,510,364-6381,yes,no,0,213.8,105,36.35,...,84,13.57,139.2,137,6.26,5.0,10,1.35,2,False.


In [315]:
df = pd.read_csv('churn.csv')
print(df.dtypes)

State              object
Account Length      int64
Area Code           int64
Phone              object
Int&#39;l Plan     object
VMail Plan         object
VMail Message       int64
Day Mins          float64
Day Calls           int64
Day Charge        float64
Eve Mins          float64
Eve Calls           int64
Eve Charge        float64
Night Mins        float64
Night Calls         int64
Night Charge      float64
Intl Mins         float64
Intl Calls          int64
Intl Charge       float64
CustServ Calls      int64
Churn?             object
dtype: object


In [316]:
def readData1(path):
    """
    Parameters
    --------------------------
        path: path of database D.
         
    --------------------------
    Returns
        data: a dictionary for representing database D
                 - keys: transaction tids
                 - values: itemsets.
        s: support of distict items in D.
    """
    data={}
    s=defaultdict(lambda: 0)
    with open(path, 'rt') as f:
        tid=1;
        header = f.readline()
        columns = header[:-1].split(',') 
        for line in f:
            line = line[:-1] 
            itemset = set(list("{}_{}".format(col, value) for col, value in zip(columns, line.split(","))))
            for item in itemset:
                s[item]+=1
            data[tid]=itemset
            tid+=1  
    return data, s

In [317]:
def joinset(a, b):
    '''
    Parameters
    -------------------
        2 itemsets a and b (of course they are at same branch in search space)
    -------------------
    return
        ret: itemset generated by joining a and b
    '''
    # TODO (hint: this function will be called in generateSearchSpace method.):
    ret = list(set(a + b))
    ret = sorted(ret)  
    return ret

class TP:
    def __init__(self, data=None, s=None, minSup=None):
        self.data = data
        self.s = {}
        for key, support in sorted(s.items(), key=lambda item: item[1]):
            self.s[key] = support
        # TODO: why should we do this, answer it at the markdown below?
        self.minSup = minSup
        self.L = {}  # Store frequent itemsets mined from database
        self.runAlgorithm()
        
    def initialize(self):
        """
        Initialize search space at first step
        --------------------------------------
        We represent our search space in a tree structure
        """
        tree = {}
        search_space = {}
        for item, support in self.s.items():
            search_space[item] = {}
            search_space[item]['itemset'] = [item]
            ''' 
            python set does not remain elements order
            so we use a list to extend it easily when create new itemset 
            but why we store itemset in data by a python set???? '''
            # TODO: study about python set and its advantages,
            # answer at the markdown below.
            search_space[item]['pruned'] = False
            # TODO:
            # After finish implementing the algorithm tell me why should you use this
            # instead of delete item directly from search_space and tree.
            search_space[item]['support'] = support
            tree[item] = {}
            '''
            Why should i store an additional tree (here it called tree)? 
            Answer: This really help in next steps.
            Remember that there is always a big gap from theory to practicality
            and implementing this algorithm in python is not as simple as you think.
            '''
        return tree, search_space
    
    def computeItemsetSupport(self, itemset):
        '''Return support of itemset'''
        support = 0
        if isinstance(itemset, int):
            itemset = {itemset}
        else:
            itemset = set(itemset)
        for transaction in self.data.values():
            if itemset.issubset(transaction):
                support += 1
        return support
    
    def get_sub_tree(self, k, tree, search_space, itter_node):
        if k == 0:
            return search_space[itter_node]['support']
        subtree = search_space[itter_node]
        for node in subtree.keys():
            k-=1
            self.get_sub_tree(k,tree,search_space,node)

    def prune(self, k, tree, search_space):
        '''
        In this method we will find out which itemset in current search space is frequent
        itemset then add it to L[k]. In addition, we prune those are not frequent itemsets.
        '''
        if self.L.get(k) is None: self.L[k] = []
        # TODO
        itemset_support = []
        for it_set in search_space.keys():
            support = self.computeItemsetSupport(it_set)
            itemset_support.append(support)
        for itemset, properties in search_space.items():  
            if properties['pruned']:
                continue
            support = self.computeItemsetSupport(search_space[itemset]['itemset'])
            if support < self.minSup:
                search_space[itemset]['pruned'] = True
            else:
                frequent_itemset = list(properties['itemset'])
                self.L[k].append(frequent_itemset)
        
    def generateSearchSpace(self, k, tree, search_space):
        '''
        Generate search space for exploring k+1 itemset. (Recursive function)
        '''
        items = list(tree.keys())
        ''' print search_space.keys() you will understand  
         why we need an additional tree, '''
        l = len(items)
        self.prune(k, tree, search_space)
        if l == 0: return  # Stop condition
        for i in range(l - 1):
            sub_search_space = {}
            sub_tree = {}
            a = items[i]
            if search_space[a]['pruned']: continue
            for j in range(i + 1, l):
                b = items[j]
                search_space[a][b] = {}
                tree[a][b] = {}
                # You really need to understand what am i doing here before doing work below.
                # TODO:
                # First create newset using join set               
                newset = tuple(sorted(joinset(list(search_space[a]['itemset']), list(search_space[b]['itemset']))))               
                # Second add newset to search_space
                if newset is not None:
                    newset_support = self.computeItemsetSupport(newset)
                    newset = tuple(newset)   
                    if newset_support >= self.minSup:
                        search_space[a][b]['itemset'] = list(newset)
                        search_space[a][b]['pruned'] = False
                        search_space[a][b]['support'] = newset_support
                        search_space[newset] = {'itemset': newset, 'pruned': False, 'support': newset_support}
                        sub_search_space[newset] = {'itemset': newset, 'pruned': False, 'support': newset_support}
                        sub_tree[newset] = {}                
            #  Generate search_space for k+1-itemset
            self.generateSearchSpace(k + 1, sub_tree, sub_search_space)
            
    def runAlgorithm(self):
        tree, search_space = self.initialize()  # generate search space for 1-itemset
        self.generateSearchSpace(1, tree, search_space)
        
    def miningResults(self):
        return self.L
    # Read churn.csv
    data1, s1 = readData1('churn.csv')
    a1=TP(data=data1,s=s1, minSup=1000)
    frequent_itemsets = a1.miningResults()
    print(frequent_itemsets)

{1: [['CustServ Calls_1'], ['Area Code_415'], ['VMail Plan_no'], ['VMail Message_0'], ['Churn?_False.'], ['Int&#39;l Plan_no']], 2: [['Churn?_False.', 'CustServ Calls_1'], ['CustServ Calls_1', 'Int&#39;l Plan_no'], ['Area Code_415', 'VMail Plan_no'], ['Area Code_415', 'VMail Message_0'], ['Area Code_415', 'Churn?_False.'], ['Area Code_415', 'Int&#39;l Plan_no'], ['VMail Message_0', 'VMail Plan_no'], ['Churn?_False.', 'VMail Plan_no'], ['Int&#39;l Plan_no', 'VMail Plan_no'], ['Churn?_False.', 'VMail Message_0'], ['Int&#39;l Plan_no', 'VMail Message_0'], ['Churn?_False.', 'Int&#39;l Plan_no']], 3: [['Area Code_415', 'VMail Message_0', 'VMail Plan_no'], ['Area Code_415', 'Int&#39;l Plan_no', 'VMail Plan_no'], ['Area Code_415', 'Int&#39;l Plan_no', 'VMail Message_0'], ['Area Code_415', 'Churn?_False.', 'Int&#39;l Plan_no'], ['Churn?_False.', 'VMail Message_0', 'VMail Plan_no'], ['Int&#39;l Plan_no', 'VMail Message_0', 'VMail Plan_no'], ['Churn?_False.', 'Int&#39;l Plan_no', 'VMail Plan_no'

- With Minup = 1000, I can realize that these result has Churn key:

['Area Code_415', 'Churn?_False.', 'Int&#39;l Plan_no'], 


['Churn?_False.', 'VMail Message_0', 'VMail Plan_no'], 


['Int&#39;l Plan_no', 'VMail Message_0', 'VMail Plan_no'], 


['Churn?_False.', 'Int&#39;l Plan_no', 'VMail Plan_no'], 


['Churn?_False.', 'Int&#39;l Plan_no', 'VMail Message_0']], 


['Churn?_False.', 'Int&#39;l Plan_no', 'VMail Message_0', 'VMail Plan_no']]}

- We can find out meaningful association law that :

If 'Int&#39;l Plan is **No** and VMail Plan is **No** that Churn = **False**

OR

If 'Int&#39;l Plan is **No** and VMail Message is **No** that Churn = **False**



# 4 References

Feel free to send questions to my email address: ntthuhang0131@gmail.com
